# Imports

In [72]:
import numpy as np
import geopandas as gpd
import tensorflow as tf
import tensorflow.keras as keras
import libpysal.weights as weights
import pysal.explore as esda

# Set random seed

In [73]:
tf.random.set_seed(42)
np.random.seed(42)

# Load data

In [74]:
raw_df = gpd.read_file("datasets/3_combined/df.gpkg")

# Separate features

In [75]:
features = raw_df.copy()
features["x_coord"] = features["geometry"].centroid.x
features["y_coord"] = features["geometry"].centroid.y
features = features.drop(columns=["geometry", "total"])
labels = features.pop("very_good_health")

# Build model

In [76]:
# Build model
normaliser = keras.layers.Normalization(axis=-1)
normaliser.adapt(np.array(features))

model = keras.Sequential([
    normaliser,
    keras.layers.Dense(372, activation="relu"),
    keras.layers.Dense(208, activation="relu"),
    keras.layers.Dense(173, activation="relu"),
    keras.layers.Dense(1)
])

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.015686), loss="mae"
)

# Fit model
early_stopper = keras.callbacks.EarlyStopping(
    monitor="val_loss", patience=20, restore_best_weights=True
)

model.fit(
    features,
    labels,
    batch_size=30,
    epochs=200,
    validation_split=0.2,
    callbacks=[early_stopper],
    verbose=1,
)

Epoch 1/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - loss: 0.3440 - val_loss: 0.0354
Epoch 2/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0257 - val_loss: 0.0316
Epoch 3/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0233 - val_loss: 0.0281
Epoch 4/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0223 - val_loss: 0.0254
Epoch 5/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0226 - val_loss: 0.0264
Epoch 6/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0224 - val_loss: 0.0301
Epoch 7/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0216 - val_loss: 0.0256
Epoch 8/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0203 - val_loss: 0.0286
Epoch 9/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0220 - val_loss: 0.0263
Epoch 10/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0202 - val_loss: 0.0261
Epoch 11/200
125/125 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 0.0217 - val_loss: 0.0340
Epoch 12/200
125/125 ━━━━━━━━━━━━━━━━━━━━

# Calculate spatial autocorrelation of residuals

In [77]:
predictions = model.predict(features).flatten()
residuals = labels - predictions
features["residuals"] = residuals
w = weights.KNN.from_dataframe(raw_df, k=8)
moran = esda.esda.Moran(features["residuals"], w)

146/146 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [78]:
moran.I

np.float64(0.21742620935404272)